# Custom metrics

In an attempt to improve the [revamped](https://github.com/Gearlux/football-predictor/blob/master/revamped.ipynb) code,
we are going to implement a custom metric which measures the profit. In order to do this, we need to supply the odds
to the keras metrics. I have not found a way to easily do this, but we can encode the odds and implement our own loss
function which can handle the encoded labels. 

It turns out that these encoded labels can also be used to write a custom loss function, which optimizes the return of our betting. 

In addition, one could also think of a way not to bet on certain matches. This is also modeled with a fourth class and a new custom loss function.

## Load the data and setup a keras environment

In [1]:
import dataset
import keras

import pandas as pd
import numpy as np
import tensorflow as tf

from keras import regularizers
from keras import metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adagrad
from keras.utils import np_utils
from keras_tqdm import TQDMNotebookCallback

from sklearn.preprocessing import StandardScaler, LabelEncoder

from keras import backend as K

np.set_printoptions(suppress=True)

book = dataset.Dataset('data/book.csv')
df = pd.DataFrame(book.processed_results)

def get_feables(df):
    features = df.drop(columns=['result'])
    encoder = LabelEncoder()
    labels = np_utils.to_categorical(-encoder.fit_transform(df.result.copy()) + 2)
    return features, labels

TRAINING_SET_FRACTION = 0.95
train_results_len = int(TRAINING_SET_FRACTION * df.shape[0])

features, labels = get_feables(df)
train_features = features[:train_results_len]
test_features = features[train_results_len:]
y_train = labels[:train_results_len]
y_test = labels[train_results_len:]

scaler = StandardScaler()
X_train = scaler.fit_transform(train_features.astype(float))
X_test = scaler.transform(test_features.astype(float))

Using TensorFlow backend.


### Utilities

To make our live easier, we are going to implement a function which constructs our model with a specified
- loss function
- a list of metrics

We also want to have reproducible results. 
With the construction of each model we are setting the seed of some libraries.

In [2]:
from numpy.random import seed
from tensorflow import set_random_seed
    
def construct_model(loss='categorical_crossentropy', metrics=['accuracy']):
    K.clear_session()

    seed(42)
    set_random_seed(42)

    model = Sequential()
    model.add(Dense(10, input_dim=X_train.shape[1],
                    kernel_regularizer=regularizers.l1(0.001)
                   ))
    model.add(Dense(3, input_dim=X_train.shape[1],
                    kernel_regularizer=regularizers.l1(0.001),
                    activation = 'softmax'
                   ))
       
    model.compile(loss=loss, 
                  optimizer=Adagrad(0.1),
                  metrics=metrics
                 )

    return model

In our [previous code](https://github.com/Gearlux/football-predictor/blob/master/revamped.ipynb), 
we have implemented a performance measure, relative to the odds of the bookmakers.

You could also think of an absolute metric which does not use the bookmakers odds for comparison,
but use the absolute probability of the model. 

In [3]:
def performance(th=0.05, features=test_features, name=None):
    def perf_metric(y_pred, y_true):
        odds = features[['odds-home','odds-draw','odds-away']]
        selection = (y_pred > (1./odds + th))
        profit = ( selection * (odds * y_true - 1)).sum()
        count = selection.sum()
        if name:
            profit.name = 'Profit ' + name
            count.name = 'Count ' + name
        return profit, count
    return perf_metric

def absolute_performance(th=0.9, features=test_features, name=None):
    def abs_perf_metric(y_pred, y_true):
        odds = features[['odds-home','odds-draw','odds-away']]
        selection = (y_pred > th)
        profit = ( selection * (odds * y_true - 1)).sum()
        count = selection.sum(axis=0)
        count = pd.Series(count, index=profit.index)
        if name:
            profit.name = '|Profit| ' + name
            count.name = '|Count| ' + name
        return profit, count
    return abs_perf_metric

## Original results


In [4]:
import shutil
import os
if os.path.exists('logs'):
    shutil.rmtree('logs')

In [5]:
model = construct_model()
_ = model.fit(X_train, y_train,
      epochs=200,
      batch_size=500, verbose=0,
      validation_data = [X_test, y_test],
      callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/original', write_graph=True), 
                 TQDMNotebookCallback(show_inner=False)]
 )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [6]:
original = pd.concat([pd.concat(performance(0.05,name='original')(model.predict(X_test), y_test),axis=1),
pd.concat(absolute_performance(0.9,name='original')(model.predict(X_test), y_test),axis=1)], axis=1)
display(original)

,Profit original,Count original,|Profit| original,|Count| original
odds-home,8.00,11,0.62,5
odds-draw,-2.00,2,0.00,0
odds-away,-0.84,13,-0.32,4


## Custom loss function

This custom loss function does not use the categorical encoded labels, 
but labels to also include the odds associated with the winning label.
We will call the labels with the encoded winning odds as `b_train` or `b_test`.

In [7]:
odds_train = train_features[['odds-home','odds-draw','odds-away']]
odds_test = test_features[['odds-home','odds-draw','odds-away']]

b_train = odds_train * (2 * y_train - 1)
b_test = odds_test * (2 * y_test - 1)

The new labels new look like

In [8]:
b_train.head()

,odds-home,odds-draw,odds-away
0,1.17,-6.5,-21.0
1,-2.00,3.3,-4.0
2,-1.91,-3.4,4.2
3,8.00,-4.5,-1.4
4,3.50,-3.3,-2.1


So the winning label has a positive odd and the losing labels have negative odds.

Our loss function should be capable of dealing with our now labels.

In [9]:
_EPSILON = 10e-8

def cat_loss(b_true, y_pred):
    prob_true = K.clip(b_true, 0., 1.)
    prob = K.clip(y_pred, _EPSILON, 1. - _EPSILON)
    res2 = K.sum(prob_true * -K.log(prob), axis=-1)
    return res2

Let's create a new model which optimizes our new loss function.

In [10]:
if os.path.exists('logs/cat_loss'):
    shutil.rmtree('logs/cat_loss')
    
model = construct_model(cat_loss)
_ = model.fit(X_train, b_train,
      epochs=200,
      batch_size=500, verbose=0,
      validation_data = [X_test, b_test],
      callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/cat_loss', write_graph=True), 
                 TQDMNotebookCallback(show_inner=False)]
 )

We also need to modify our performance functions.

In [11]:
def b_performance(th=0.05, name=None):
    def perf_metric(y_pred, y_true):
        odds = np.abs(y_true)
        selection = (y_pred > (1./odds + th))
        profit = ( selection * (odds * np.clip(y_true, 0, 1) - 1)).sum(axis=0)
        count = selection.sum(axis=0)
        profit = pd.Series(profit)
        profit.index= ['home','draw','away']
        count = pd.Series(count, index=profit.index)
        if name:
            profit.name = 'Profit ' + name
            count.name = 'Count ' + name
        return profit, count
    return perf_metric

def b_absolute_performance(th=0.9, name=None):
    def abs_perf_metric(y_pred, y_true):
        odds = np.abs(y_true)

        selection = (y_pred > th)
        profit = ( selection * (odds * np.clip(y_true, 0, 1) - 1)).sum(axis=0)
        count = selection.sum(axis=0)
        profit = pd.Series(profit)
        profit.index= ['home','draw','away']
        count = pd.Series(count, index=profit.index)
        if name:
            profit.name = '|Profit| ' + name
            count.name = '|Count| ' + name
        return profit, count
    return abs_perf_metric

In [12]:
b = pd.concat([pd.concat(b_performance(0.05,name='b')(model.predict(X_test), b_test.values),axis=1),
pd.concat(b_absolute_performance(0.9,name='b')(model.predict(X_test), b_test.values),axis=1)], axis=1)
display(b)

,Profit b,Count b,|Profit| b,|Count| b
home,8.00,11,0.62,5
draw,-2.00,2,0.00,0
away,-0.84,13,-0.32,4


If we have implemented the `cat_loss` function correctly, we should have similar results as the original profit.

In [13]:
display(original)

,Profit original,Count original,|Profit| original,|Count| original
odds-home,8.00,11,0.62,5
odds-draw,-2.00,2,0.00,0
odds-away,-0.84,13,-0.32,4


Success! Because we have setup the seeds of both our libraries to the answer to the universe,
we obtain exaclty the same result.

## Better metrics

Don't forget the reason why we want to implement something similar.

The reason why we want to implement this new loss function, is to define a metric which can shows us the
profit.
In the previous section, we could only monitor the profit at the end of fitting process.
It makes more sense to monitor the profit during the learning process.

We implemented two performance metrics in the previous section.
Likewise, we will also implement two metrics who will each implement both the profit and the margin which 
can be monitored in tensorboard.

In [14]:
def bet_metric(th=0.05):
    def profit(y_true, y_pred):
        true_odds = K.abs(y_true)
        selection = K.cast((y_pred > (1./true_odds + th)),'float32')
        odds_true = K.clip(y_true, 0., np.inf) - 1
        return K.sum(K.sum( selection * odds_true))
    def margin(y_true, y_pred):
        true_odds = K.abs(y_true)
        selection = K.cast((y_pred > (1./true_odds + th)),'float32')
        odds_true = K.clip(y_true, 0., np.inf) - 1
        return K.sum(K.sum( selection * odds_true)) / K.sum(K.sum(selection))
    return profit, margin

def abs_bet_metric(th=0.9):
    def abs_profit(y_true, y_pred):
        selection = K.cast((y_pred > th),'float32')
        odds_true = K.clip(y_true, 0., np.inf) - 1
        return K.sum(K.sum( selection * odds_true))
    def abs_margin(y_true, y_pred):
        selection = K.cast((y_pred > th),'float32')
        odds_true = K.clip(y_true, 0., np.inf) - 1
        return K.sum(K.sum( selection * odds_true)) / K.sum(K.sum(selection))
    return abs_profit, abs_margin

In [15]:
if os.path.exists('logs/bet_metric'):
    shutil.rmtree('logs/bet_metric')
    
model = construct_model(cat_loss, metrics=['accuracy'] + list(bet_metric(0.05)) + list(abs_bet_metric(0.9)))
_ = model.fit(X_train, b_train,
      epochs=200,
      batch_size=500, verbose=0,
      validation_data = [X_test, b_test],
      callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/bet_metric', write_graph=True), 
                 TQDMNotebookCallback(show_inner=False)]
 )

In [16]:
b = pd.concat([pd.concat(b_performance(0.05,name='b')(model.predict(X_test), b_test.values),axis=1),
pd.concat(b_absolute_performance(0.9,name='b')(model.predict(X_test), b_test.values),axis=1)], axis=1)
display(b)

,Profit b,Count b,|Profit| b,|Count| b
home,8.00,11,0.62,5
draw,-2.00,2,0.00,0
away,-0.84,13,-0.32,4


These are the same results as in the previous run, and surprise, surprise !
In tensorboard, we can find the same end result for the profits and derived margins.
Not only do we have the metric for our training set, but it's also available for the validation set.

<tr>
    <td><img src="images/acc.png" width="400px"></td>
    <td><img src="images/loss.png" width="400px"></td>
</tr>
<tr>
    <td><img src="images/profit.png" width="400px"></td>
    <td><img src="images/val_profit.png" width="400px"></td>
</tr>

But the perfomance and the metrics, are similar functions. Can we share the code ?

In [18]:
import tensorflow as tf

def profit_(y_true, y_pred, threshold):
    selection = K.cast((y_pred > threshold),'float32')
    odds_true = K.clip(y_true, 0., np.inf) - 1
    return K.sum( selection * odds_true, axis=0), K.sum(selection, axis=0)

def bet_metric(th=0.05):
    def profit(y_true, y_pred):
        true_odds = K.abs(y_true)
        condition = (1./true_odds + th)
        return K.sum(profit_(y_true, y_pred, condition)[0])
    def margin(y_true, y_pred):
        true_odds = K.abs(y_true)
        condition = (1./true_odds + th)
        pr = profit_(y_true, y_pred, condition)
        return K.sum(pr[0]) / K.sum(pr[1])
    return profit, margin

def abs_bet_metric(th=0.9):
    def abs_profit(y_true, y_pred):
        return K.sum(profit_(y_true, y_pred, th)[0])
    def abs_margin(y_true, y_pred):
        pr = profit_(y_true, y_pred, th)
        return K.sum(pr[0]) / K.sum(pr[1])
    return abs_profit, abs_margin

def b_performance(th=0.05, name=None):
    def perf_metric(y_pred, y_true):
        ypred = tf.convert_to_tensor(y_pred, dtype=tf.float32)
        ytrue = tf.convert_to_tensor(y_true, dtype=tf.float32)
        true_odds = K.abs(ytrue)
        condition = (1./true_odds + th)
        pr, co = profit_(ytrue, ypred, condition)
        idx = ['home','draw','away']
        profit = pd.Series(K.eval(pr), index=idx)
        count = pd.Series(K.eval(co), index=idx)
        if name:
            profit.name = 'Profit ' + name
            count.name = 'Count ' + name
        return profit, count
    return perf_metric

def b_absolute_performance(th=0.9, name=None):
    def abs_perf_metric(y_pred, y_true):
        ypred = tf.convert_to_tensor(y_pred, dtype=tf.float32)
        ytrue = tf.convert_to_tensor(y_true, dtype=tf.float32)
        pr, co = profit_(ytrue, ypred, th)
        idx = ['home','draw','away']
        profit = pd.Series(K.eval(pr), index=idx)
        count = pd.Series(K.eval(co), index=idx)
        if name:
            profit.name = '|Profit| ' + name
            count.name = '|Count| ' + name
        return profit, count
    return abs_perf_metric

In [19]:
if os.path.exists('logs/keras_metric'):
    try:
        shutil.rmtree('logs/keras_metric')
    except OSError:
        pass
    
model = construct_model(cat_loss, metrics=['accuracy'] + list(bet_metric(0.05)) + list(abs_bet_metric(0.9)))
_ = model.fit(X_train, b_train,
      epochs=200,
      batch_size=500, verbose=0,
      validation_data = [X_test, b_test],
      callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/keras_metric', write_graph=True), 
                 TQDMNotebookCallback(show_inner=False)]
 )

In [20]:
cm = pd.concat([pd.concat(b_performance(0.05,name='cm')(model.predict(X_test), b_test.values),axis=1),
pd.concat(b_absolute_performance(0.9,name='cm')(model.predict(X_test), b_test.values),axis=1)], axis=1)
display(cm)

,Profit cm,Count cm,|Profit| cm,|Count| cm
home,8.00,11.0,0.62,5.0
draw,-2.00,2.0,0.00,0.0
away,-0.84,13.0,-0.32,4.0


### Intermission

With our `cat_loss`, `bet_metric` and `abs_bet_metric` we now have tools to better monitor 

In [21]:
def clean_folds():
    for i in range(100):
        if os.path.exists('logs/fold_%02d' % i):
            try:
                shutil.rmtree('logs/fold_%02d' % i)
            except (PermissionError, OSError):
                pass
clean_folds()

In [22]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=20, random_state=42)

def cross_validate(epochs=200, loss=cat_loss):
    results = []
    models = []

    for i, (trainidx, valididx) in enumerate(kfold.split(df)):
        train = df.iloc[trainidx]
        test = df.iloc[valididx]
        train_features = train.drop(columns=['result'])
        train_labels = train.result.copy()
        test_features = test.drop(columns=['result'])
        test_labels = test.result.copy()
        scaler = StandardScaler()
        X_train = scaler.fit_transform(train_features.astype(float))
        X_test = scaler.transform(test_features.astype(float))
        encoder = LabelEncoder()
        Y_train = -encoder.fit_transform(train_labels) +2
        Y_test = -encoder.transform(test_labels) +2
        y_train = np_utils.to_categorical(Y_train)
        y_test = np_utils.to_categorical(Y_test)    
        odds_train = train_features[['odds-home','odds-draw','odds-away']]
        odds_test = test_features[['odds-home','odds-draw','odds-away']]
        c_train = odds_train * (2 * y_train - 1)
        c_test = odds_test * (2 * y_test - 1)

        model = construct_model(loss, metrics=['accuracy'] + list(bet_metric(0.05)) + list(abs_bet_metric(0.9)))

        _ = model.fit(X_train, c_train,
              epochs=epochs,
              batch_size=500, verbose=0,
              validation_data = [X_test, c_test],
              callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/fold_%02d' % i, write_graph=True), 
                         TQDMNotebookCallback(show_inner=False)]
         )    

        cm = pd.concat([pd.concat(b_performance(0.05,name='cm')(model.predict(X_test), c_test.values),axis=1),
        pd.concat(b_absolute_performance(0.9,name='cm')(model.predict(X_test), c_test.values),axis=1)], axis=1)
        results.append(cm)
        models.append(model)

        display(cm)
        
    return results, models

In [23]:
results, models = cross_validate()

,Profit cm,Count cm,|Profit| cm,|Count| cm
home,5.54,17.0,0.59,4.0
draw,6.15,7.0,0.00,0.0
away,0.46,5.0,0.00,0.0


,Profit cm,Count cm,|Profit| cm,|Count| cm
home,2.43,7.0,0.14,1.0
draw,5.30,16.0,0.00,0.0
away,0.30,1.0,0.00,0.0


,Profit cm,Count cm,|Profit| cm,|Count| cm
home,2.25,6.0,0.13,1.0
draw,0.40,6.0,0.00,0.0
away,-1.57,7.0,-0.71,2.0


,Profit cm,Count cm,|Profit| cm,|Count| cm
home,0.75,10.0,0.34,3.0
draw,9.35,11.0,0.00,0.0
away,4.52,3.0,0.22,1.0


,Profit cm,Count cm,|Profit| cm,|Count| cm
home,6.93,12.0,0.13,1.0
draw,1.60,2.0,0.00,0.0
away,0.00,0.0,0.00,0.0


...

,Profit cm,Count cm,|Profit| cm,|Count| cm
home,0.76,24.0,0.0,0.0
draw,3.75,1.0,0.0,0.0
away,0.95,2.0,0.0,0.0


,Profit cm,Count cm,|Profit| cm,|Count| cm
home,0.3,18.0,0.0,0.0
draw,0.0,0.0,0.0,0.0
away,3.6,6.0,0.0,0.0


,Profit cm,Count cm,|Profit| cm,|Count| cm
home,-1.0,1.0,0.0,0.0
draw,-2.5,6.0,0.0,0.0
away,0.0,10.0,0.0,0.0


,Profit cm,Count cm,|Profit| cm,|Count| cm
home,-2.92,15.0,0.9,7.0
draw,-3.00,3.0,0.0,0.0
away,-0.68,6.0,-1.0,1.0


,Profit cm,Count cm,|Profit| cm,|Count| cm
home,3.50,10.0,0.62,5.0
draw,-2.00,2.0,0.00,0.0
away,-2.29,16.0,-0.32,4.0


Examine the profit of our folded model.

In [24]:
result = pd.concat({n: df for n, df in enumerate(results)},axis=0)
result.sum(level=1)

,Profit cm,Count cm,|Profit| cm,|Count| cm
home,14.639999,269.0,2.02,32.0
draw,23.100000,96.0,0.00,0.0
away,15.290000,85.0,-1.81,8.0


There is a small \(positive\) difference with our 
[revamped](https://github.com/Gearlux/football-predictor/blob/master/revamped.ipynb)
model.
We now have a 11% margin for our relative model and a 0.5% margin for our absolute threshold.
The difference with the [revamped](https://github.com/Gearlux/football-predictor/blob/master/revamped.ipynb)
model are the number of epochs and the random seed.

Maybe the evolution of our metrics gain some insight.

<tr>
<td>
    <img src="images/margin_folds.png" width="400px"></td>
    <td><img src="images/profit_folds.png" width="400px"></td>
</tr>


We see that margin for all our folds is increasing, but the profit stays more ore less constant.

Inspection of the validation folds gives a diffent view. 
This might be an indication that we are overfitting the model or that our model is not capable of modeling
the complex interactions for betting.

Optimization of the network structure, will be investigated in a seperate project.

<tr>
<td>
    <img src="images/val_margin_folds.png" width="400px"></td>
    <td><img src="images/val_profit_folds.png" width="400px"></td>
</tr>